In [73]:
import gdal
import osr
import os


os.environ['PROJ_LIB'] = 'proj/'

In [74]:
# 地理坐标路径
file_name = '2022-07-13_3_S2DL.tif'
raster_path = r'E:\Desktop\LAI回归模型训练\01GEE0713\{}'.format(file_name)
# 保存投影栅格数据路径
UTM_raster_path = r'E:\Desktop\LAI回归模型训练\02GEE0713Projection\1{}'.format(file_name)
longitude = 135
is_north = True

In [75]:
raster_ds = gdal.Open(raster_path)
raster_type = raster_ds.GetRasterBand(1).DataType

In [76]:
raster_ds.GetProjection()
raster_ds.RasterXSize

4659

In [77]:
# 栅格地理坐标
spatialRef = osr.SpatialReference()

In [78]:
spatialRef.ImportFromWkt(raster_ds.GetProjection())
spatialRef

<osgeo.osr.SpatialReference; proxy of <Swig Object of type 'OSRSpatialReferenceShadow *' at 0x0000029B9F3A3D80> >

In [79]:
# 根据经度计算UTM区号,进而定义UTM投影
zone = str(int(longitude / 6) + 31)
zone = int('326' + zone) if is_north else int('327' + zone)
UTM_spatialRef = osr.SpatialReference()
UTM_spatialRef.ImportFromEPSG(zone)

# 投影转换
coordinate_transfor = osr.CoordinateTransformation(spatialRef, UTM_spatialRef)
coordinate_transfor

<osgeo.osr.CoordinateTransformation; proxy of <Swig Object of type 'OSRCoordinateTransformationShadow *' at 0x0000029B9E2DA5A0> >

In [80]:
# 仿射矩阵六参数
geotransform = raster_ds.GetGeoTransform()
# 左上角upper left、右下角lower right坐标

geotransform

(132.91275314469934,
 8.983152841195215e-05,
 0.0,
 47.43176565373803,
 0.0,
 -8.983152841195215e-05)

In [81]:
ul_x = geotransform[0]
ul_y = geotransform[3]
lr_x = geotransform[0] + geotransform[1] * raster_ds.RasterXSize + geotransform[2] * raster_ds.RasterYSize
lr_y = geotransform[3] + geotransform[4] * raster_ds.RasterYSize + geotransform[5] * raster_ds.RasterYSize

In [82]:
# 左上角、右下角在目标投影中的坐标
(UTM_ul_x, UTM_ul_y, UTM_ul_z) = coordinate_transfor.TransformPoint(ul_y, ul_x)
(UTM_ul_x, UTM_ul_y, UTM_ul_z)

(342601.62630853354, 5255258.363241494, 0.0)

In [83]:
(UTM_lr_x, UTM_lr_y, UTM_lr_z) = coordinate_transfor.TransformPoint(lr_y, lr_x)

(UTM_lr_x, UTM_lr_y, UTM_lr_z)

(373564.98364082444, 5226634.10626176, 0.0)

In [84]:
# 创建目标图像文件
driver = gdal.GetDriverByName("GTiff")
UTM_raster_ds = driver.Create(UTM_raster_path,
                              raster_ds.RasterXSize,
                              raster_ds.RasterYSize,
                              raster_ds.RasterCount,
                              raster_type)
# 转换后图像的分辨率
# resolution = (UTM_lr_x - UTM_ul_x) / raster_ds.RasterXSize
resolution = 10

In [85]:
# 转换后图像的六个放射变换参数
UTM_transform = [UTM_ul_x, resolution, 0, UTM_ul_y, 0, -resolution]
UTM_transform

[342601.62630853354, 10, 0, 5255258.363241494, 0, -10]

In [86]:
UTM_raster_ds.SetGeoTransform(UTM_transform)
UTM_raster_ds.SetProjection(UTM_spatialRef.ExportToWkt())
# 投影转换后需要做重采样
gdal.ReprojectImage(raster_ds, UTM_raster_ds, spatialRef.ExportToWkt(),
                    UTM_spatialRef.ExportToWkt(), gdal.GRA_Bilinear)

0

In [87]:
# 关闭
raster_ds = None
UTM_raster_ds = None